# Sponges classification using Yolo v4 on Google Colab
Based on https://github.com/quangnhat185/darknet_for_colab

## General parameters

In [ ]:
# Parameters

IS_A_NEW_TRAINING = False
DATASET_VERSION = "v10"
TRAIN_ID = "001"
DRIVE_FOLDER = "YOLOv4_sponges"

## Mount with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup environment

In [ ]:
# Imports
print("Importing dependancies...")
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image, ImageDraw
import glob
!pip install opencv-python==4.5.1.48 --force-reinstall > /dev/null &> /dev/null
import cv2
import numpy as np

# Check whether GPU is provided
print("GPU Driver :")
!nvcc --version
print()


# Download darknet
print("Downloading darknet_for_colab repository...")
!git clone https://github.com/quangnhat185/darknet_for_colab
%cd darknet_for_colab
print()


# Compile darknet
print("Compiling darknet...")
!make &> compilation_report.txt
!chmod +x ./darknet
print()


# Download YoloV4 pre-trained weights
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())
if IS_A_NEW_TRAINING:
  print("Downloading pre-trained weights...")
  !wget -q https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
  print()


# Download sponge dataset
%cd data
assert os.getcwd()=='/content/darknet_for_colab/data', 'Directory should be "/content/darknet_for_colab/data" instead of "{}"'.format(os.getcwd())

print("Downloading dataset...")
!wget -q --no-check-certificate "https://www.dev.alexandre-thomas.fr/datasets/sponges_dataset_"$DATASET_VERSION".zip" -O data.zip
!unzip -q data.zip
!rm -f data.zip
%cd ..
print()


# Visualize sample of the dataset
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

def read_label(image_path):
  file_name = image_path.replace('.jpg', '.txt')
  with open(file_name, 'rt') as file:
    print(os.path.basename(file_name) + ': \n' + file.read())

print("Generating sample of the dataset...")
image_path = glob.glob("data/data/*.jpg")
fig = plt.figure(figsize=(12,8))
cols = 2
rows = 2
grid = gridspec.GridSpec(nrows=rows, ncols=cols, figure=fig)
for i in range(cols*rows):
  fig.add_subplot(grid[i])
  image=plt.imread(image_path[i])
  plt.title(os.path.basename(image_path[i]))
  plt.axis(False)
  plt.imshow(image)
  read_label(image_path[i])

plt.savefig("dataset_examples.jpg", dpi=300)
print()


# Edit YoloV4 hyperparameters
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# Run python script to create customize yolov4_custom_train.cfg and yolov4_custom_test.cfg in folder /cfg
print("Creating custom config files...")
!python yolov4_setup.py
print()


# Create symbolic link in drive
print("Creating symbolic link to GDrive...")

!rmdir backup
!rmdir charts

if IS_A_NEW_TRAINING:
  !mkdir "/content/drive/My Drive/"$DRIVE_FOLDER"/"$TRAIN_ID

  !mkdir "/content/drive/My Drive/"$DRIVE_FOLDER"/"$TRAIN_ID"/backup"
  !mkdir "/content/drive/My Drive/"$DRIVE_FOLDER"/"$TRAIN_ID"/charts"

!ln -s "/content/drive/My Drive/"$DRIVE_FOLDER"/"$TRAIN_ID"/backup" /content/darknet_for_colab
!ln -s "/content/drive/My Drive/"$DRIVE_FOLDER"/"$TRAIN_ID"/charts" charts
print()

## Training

In [ ]:
# $clear_this_cell$

assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

weights_path = "'/content/drive/My Drive/" + DRIVE_FOLDER + "/" + TRAIN_ID + "/backup/yolov4_custom_train_last.weights'"
if IS_A_NEW_TRAINING:
  weights_path = "yolov4.conv.137"

print("Begin training...")
!./darknet detector train data/yolov4.data cfg/yolov4_custom_train.cfg $weights_path -dont_show -map

Copy this script in the browser console to clear the previous cell every 45s
```javascript
var currentInterval = null;
function clearButton() {
    cells = document.querySelectorAll(".notebook-content textarea");
    tag = "$clear_this_cell$";
    cell = null;
    
    for(i=0; i<cells.length; i++) {
        if (cells[i].value.includes(tag))
            cell = cells[i];
    }

    el = cell.closest(".cell.code");
    id = el.id;

    button = document.querySelectorAll("#" + id + " .output-info iron-icon")[1];

    return button;
}

currentInterval = setInterval(function() {b = clearButton(); if(b){b.click();}}, 45000);
```

## Calculate mAP

In [ ]:
weights_path = "'/content/drive/My Drive/" + DRIVE_FOLDER + "/" + TRAIN_ID + "/backup/best/yolov4_custom_train_best6.weights'"
#weights_path = "'/content/drive/My Drive/" + DRIVE_FOLDER + "/" + TRAIN_ID + "/backup/Best weights/yolov4_custom_train_best1.weights'"

!./darknet detector map data/yolov4.data cfg/yolov4_custom_test.cfg $weights_path

## Test with example image

In [ ]:
print("Begin detection on test set...")

weights_path = "'/content/drive/My Drive/" + DRIVE_FOLDER + "/" + TRAIN_ID + "/backup/yolov4_custom_train_last.weights'"
!./darknet detector test data/yolov4.data cfg/yolov4_custom_test.cfg $weights_path -dont_show -thresh 0.05 < data/test.txt > results.txt

print()
print("Results :")

In [ ]:
# Based on : https://learnopencv.com/deep-learning-based-object-detection-using-yolov3-with-opencv-python-c/
# and : https://www.mygreatlearning.com/blog/yolo-object-detection-using-opencv/

import time

assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

print("OpenCV version : %s" % cv2.__version__)

!rm -rf predictions;
!mkdir -p "/content/drive/My Drive/"$DRIVE_FOLDER"/"$TRAIN_ID"/predictions"
!ln -s "/content/drive/My Drive/"$DRIVE_FOLDER"/"$TRAIN_ID"/predictions" predictions

start = time.time()

# Load names of classes
classesFile = "data/classes.names";
#classes = ["a", "b", "c", "d", "e"]
with open(classesFile, 'rt') as f:
  classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "cfg/yolov4_custom_test.cfg";
#modelWeights = "/content/drive/My Drive/" + DRIVE_FOLDER + "/" + TRAIN_ID + "/backup/yolov4_custom_train_best.weights";
modelWeights = "/content/drive/My Drive/" + DRIVE_FOLDER + "/" + TRAIN_ID + "/backup/yolov4_custom_train_1400.weights"

# Load Yolo Network
print("Loading Yolo...")
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

layer_names = net.getLayerNames()

#Determine the output layer names from the YOLO model 
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

print("Yolo loaded")


test_filenames = []
with open("data/test.txt") as f:
  test_filenames = [l.rstrip("\n") for l in f.readlines()]

colors = [
  [76, 177, 34],
  [164, 73, 163],
  [155, 105, 0],
  [0, 0, 230],
  [0, 168, 217],
  [100, 0, 0]
]

for i, filename in enumerate(test_filenames):
  print("Processing image %d/%d : %s" %(i+1, len(test_filenames), filename))

  # Load image
  img = cv2.imread(filename)
  height, width, channels = img.shape

  # Preprocess image
  blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=True, crop=False)

  # Detecting objects
  net.setInput(blob)
  outs = net.forward(output_layers)

  # Showing informations
  class_ids = []
  confidences = []
  boxes = []
  for out in outs:
    for detection in out:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]
      if confidence > 0.5:
        # Object detected
        center_x = int(detection[0] * width)
        center_y = int(detection[1] * height)
        w = int(detection[2] * width)
        h = int(detection[3] * height)

        # Rectangle coordinates
        x = int(center_x - w / 2)
        y = int(center_y - h / 2)

        boxes.append([x, y, w, h])
        confidences.append(float(confidence))
        class_ids.append(class_id)
    
  # Use NMS function in opencv to perform Non-maximum Suppression
  # Give it score threshold and NMS threshold as arguments.
  indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
  for i in range(len(boxes)):
    if i in indexes:
      x, y, w, h = boxes[i]
      label = "%s : %.2f" % (classes[class_ids[i]], confidences[i])
      color = colors[class_ids[i]]
      labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)

      cv2.rectangle(img, (x, y), (x + w, y + h), color, 6)
      cv2.rectangle(img, (x - 3, y - labelSize[1] - 20), (x + labelSize[0], y), color, -1)
      cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

  cv2.imwrite("predictions/" + filename.split("/")[-1], img)

print("Predictions complete in %s on %d images, they are available in predictions/ folder" % (str(time.time() - start) , len(test_filenames)))